In [2]:
# Importamos las librerías que necesitamos

# Librerías de extracción de datos
# -----------------------------------------------------------------------

# Importaciones:
# Beautifulsoup
from bs4 import BeautifulSoup

# Requests
import requests

import pandas as pd
import numpy as np

from time import sleep
import random
import datetime

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException # Excepciones comunes de selenium que nos podemos encontrar 

In [3]:
driver = webdriver.Chrome()
url_wunder = "https://www.wunderground.com/history"
driver.get(url_wunder)
driver.maximize_window()

#Denegamos las cookies
iframe_cookies = WebDriverWait(driver, 10).until(EC.presence_of_element_located(("xpath", '//*[@id="sp_message_iframe_1165301"]')))
driver.switch_to.frame(iframe_cookies)

sleep(5)
try:
    driver.find_element("css selector", "#notice > div.message-component.message-row.cta-buttons-container > div.message-component.message-column.cta-button-column.reject-column").click()
except:
    print("No encuentro el boton de las cookies")

driver.switch_to.default_content()
# Introduce el municipio en la página principal
sleep(3)
driver.find_element("css selector", "#historySearch").send_keys("Madrid, Madrid, Spain", Keys.ENTER)

# Damos click en view
sleep(3)
driver.find_element("css selector", "#dateSubmit").click()
driver.find_element("css selector", "#dateSubmit").click()

# Pinchar en monthly de la página del municipio
sleep(3)
driver.find_element("css selector", "#inner-content > div.region-content-main > div.row > div:nth-child(1) > div:nth-child(1) > div > lib-link-selector > div > div > div > a:nth-child(3)").click()

sleep(2)
html_table_page = driver.page_source



In [39]:
# Aqui habria que aprender a cambiar de mes y año

# driver.find_element("css selector", "#monthSelection").click()
# driver.find_element("css selector", "#monthSelection").click()


In [4]:
sopa = BeautifulSoup(html_table_page, "html.parser")


In [5]:
dias_mes = {
    "Jan": 31,
    "Feb": 28,
    "Mar": 31,
    "Apr": 30,
    "May": 31,
    "Jun": 30,
    "Jul": 31,
    "Aug": 31,
    "Sep": 30,
    "Oct": 31,
    "Nov": 30,
    "Dec": 31
}

meses = {
    1: "Jan",
    2: "Feb",
    3: "Mar",
    4: "Apr",
    5: "May",
    6: "Jun",
    7: "Jul",
    8: "Aug",
    9: "Sep",
    10: "Oct",
    11: "Nov",
    12: "Dec"
}

In [6]:
hora_actual = datetime.datetime.now().month
print(hora_actual)

10


In [21]:
def getText(linea):
    dato = linea.text
    return dato.strip()
    

In [86]:
table = sopa.findAll("table", {"class":"days ng-star-inserted"})
columnas = table[0].findAll("td")

mes_actual = columnas[8].text.strip()
dias=16
encabezados=columnas[:7]
encabezados_limpio = list(map(getText,encabezados))

listas = []
df = pd.DataFrame()

# dias
max1=8+1+dias
mes=columnas[8:max1]
listas.append(mes)

#temperatura
max2=max1+1+((dias+1)*3)
temperaturas=columnas[max1+1:max2]
listas.append(temperaturas)

#rocío
max3=max2+1+((dias+1)*3)
rocio=columnas[max2+1:max3]
listas.append(rocio)

#humedad
max4=max3+1+((dias+1)*3)
humedad=columnas[max3+1:max4]
listas.append(humedad)

#viento
max5=max4+1+((dias+1)*3)
viento=columnas[max4+1:max5]
listas.append(viento)

#presion
max6=max5+1+((dias+1)*3)
presion=columnas[max5+1:max6]
listas.append(presion)

#precipitacion
max7=max6+1+dias+1
precipitacion = columnas[max6+1:max7]
listas.append(precipitacion)

i=0
for lista in listas:
    lista_limpia = list(map(getText,lista))
    if i!=0 and i!=6:
        df_aux=pd.DataFrame(np.reshape(lista_limpia,(dias+1,3)))
    else:
        df_aux=pd.DataFrame(lista_limpia)
    df = pd.concat([df, df_aux], axis=1)
    i+=1

#Metemos el mes, renombramos las columnas, rehacemos el indice y que quitamos la columna indice
df.insert(0,"mes", mes_actual)
df.columns = ["mes","dia", "max_temp", "avg_temp", "min_temp", "max_rocio", "avg_rocio", "min_rocio", 
              "max_humedad", "avg_humedad", "min_humedad", "max_viento", "avg_viento", 
              "min_viento", "max_presion", "avg_presion", "min_presion", "lluvia"]
df = df.drop(index=0)
df.reset_index(inplace=True)
df.drop("index", axis=1, inplace=True)


In [87]:
df

,mes,dia,max_temp,avg_temp,min_temp,max_rocio,avg_rocio,min_rocio,max_humedad,avg_humedad,min_humedad,max_viento,avg_viento,min_viento,max_presion,avg_presion,min_presion,lluvia
0,Oct,1,84,66.8,50,55,45.2,41,72,48.8,23,10,3.6,0,28.0,27.9,27.9,0.00
1,Oct,2,79,71.2,63,63,57.0,52,78,62.0,47,18,8.8,0,27.9,27.8,27.7,0.00
2,Oct,3,77,67.6,61,59,49.6,37,88,55.5,24,16,5.6,0,27.9,27.8,27.7,0.00
3,Oct,4,77,63.9,50,50,46.1,43,82,53.9,36,8,2.3,0,27.9,27.9,27.8,0.00
4,Oct,5,81,65.5,50,59,51.0,45,87,61.8,39,17,5.9,0,27.9,27.9,27.8,0.00
5,Oct,6,75,69.7,64,59,56.7,55,73,64.1,50,17,12.7,1,27.8,27.8,27.8,0.00
6,Oct,7,75,63.6,57,57,53.6,48,94,71.8,38,24,6.9,0,27.8,27.7,27.6,0.00
7,Oct,8,64,60.0,55,57,54.6,52,88,82.0,68,17,11.4,3,27.8,27.8,27.7,0.00
8,Oct,9,70,63.0,59,63,57.3,50,94,81.4,63,23,12.2,3,27.8,27.6,27.5,0.00
9,Oct,10,68,60.4,52,50,43.9,41,76,55.4,37,20,7.3,0,27.9,27.8,27.7,0.00
